In [1]:
using WebIO, Interact, StatsPlots, DataFrames, CSV, Dates;
WebIO.install_jupyter_nbextension() # might need to restart Jupyter afterwards in order to work

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9090450699441963724\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-454450313757232284\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-9090450699441963724\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
Input_df = CSV.read("data/200124_datafile_all_details_right_timestamp.csv");

In [2]:
h_predict=8760;
h_control=8760;
h_start=1;
h_end=8760;
objective=1;
case=1;
costfactor=1.0;

In [4]:
Flow_df = CSV.read("results/results_$(h_predict)_$(h_control)_$(h_start)"*
    "-$(h_end)_$(objective)_$(case)_$(costfactor).csv");

In [5]:
first(Flow_df, 3)

,Temp_FH,Vol_HW,Soc_B,V_HW_plus,V_HW_minus,T_FH_plus,T_FH_minus,profits
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,22.0,180.0,13.5,-0.0,0.0,-0.0,0.0,0.0
2,22.0,179.329,11.9555,0.0,0.0,0.0,0.0,0.0
3,22.0,178.658,10.3964,0.0,0.0,0.0,0.0,0.0


In [6]:
first(Input_df, 3)

,heating,heating_fan,hotwater,lights,lgappl,vent_fan,misc,electkwh
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,3.73669,0.044709,0.0,0.022749,0.042381,0.0,0.176259,0.241389
2,3.83768,0.035999,0.0,0.013649,0.041584,0.0,0.166187,0.22142
3,6.05949,0.041142,0.0,0.009099,0.036289,0.0,0.161151,0.206539


In [61]:
# merge on data stamp
Data_df = leftjoin(Flow_df, Input_df, on=[:month, :day, :hour], makeunique=false, 
    indicator=nothing, validate=(false, false));
first(Data_df,3)

,Temp_FH,Vol_HW,Soc_B,V_HW_plus,V_HW_minus,T_FH_plus,T_FH_minus,profits
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,22.0,180.0,13.5,-0.0,0.0,-0.0,0.0,0.0
2,22.0,179.329,11.9555,0.0,0.0,0.0,0.0,0.0
3,22.0,178.658,10.3964,0.0,0.0,0.0,0.0,0.0


In [8]:
size(Data_df)

(8760, 36)

In [9]:
println.(names(Data_df));

Temp_FH
Vol_HW
Soc_B
V_HW_plus
V_HW_minus
T_FH_plus
T_FH_minus
profits
COP_FH
COP_HW
PV_DE
B_DE
GR_DE
PV_B
PV_GR
PV_HP
GR_HP
B_HP
HP_FH
HP_HW
month
day
hour
horizon
heating
heating_fan
hotwater
lights
lgappl
vent_fan
misc
electkwh
heatingkwh
hotwaterkwh
PV_generation
Temperature


In [27]:
function bar_PV(Data_df, date)
    # bars
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] groupedbar(
        [:PV_HP :PV_GR :PV_B :PV_DE], color =[:firebrick :grey :purple :orange], 
        ylim=(0,15), yticks=0:5:15, xticks=0:6:24,
        annotations=(2,12, text("$(Dates.month(date))/$(Dates.day(date))", 10)), legendfontsize=6, 
        label=["PV_HP" "PV_GR" "PV_B" "PV_DE"], legend=false, bar_position = :stack, alpha=0.8);
    # battery cap
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] plot!(
        [:Soc_B], color =[:purple], ylim=(0,15), yticks=0:2:15, xticks=0:6:24, 
        label=["SOC_b"], legend=false, linewidth= 2.0);
    # PV generation
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] plot!(
        [:PV_generation], ylim=(0,15), yticks=0:5:15, xticks=0:6:24, color =[:gold], 
        label=["g_e"], legend=false, linewidth= 2.0);
    
    plot!(0:23, ones(24,1)*13.5, linestyle=:dot, linewidth= 2, color=:purple)
end

bar_PV (generic function with 1 method)

In [35]:
function bar_demand(Data_df, date)
    # bars
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] groupedbar(
        [:B_DE :GR_DE :PV_DE ], color =[:purple :grey :gold], 
        ylim=(0,2), yticks=0:0.5:2, xticks=0:6:24,
        annotations=(2,1.8, text("$(Dates.month(date))/$(Dates.day(date))", 10)), legendfontsize=6, 
        label=["B_DE" "GR_DE" "PV_DE"], legend=false, bar_position = :stack, alpha=0.8);
    # electricity demand
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] plot!(
        [:electkwh], ylim=(0,2), yticks=0:0.5:2, xticks=0:6:24, color =[:orange], 
        label=["d_e"], legend=false, linewidth= 2.0);
    
    plot!(0:23, ones(24,1)*13.5, linestyle=:dash, color=:purple)
end

bar_demand (generic function with 2 methods)

In [37]:
function bar_heat(Data_df, date)
    # hot water demand
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] plot(
        [:hotwaterkwh]+[:heatingkwh], fillrange=[:heatingkwh], ylim=(0,15), yticks=0:5:15, xticks=0:6:24, 
        color =[:steelblue], label=["d_fh"], legend=false, alpha=0.2);
    # heat demand
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] plot!(
        [:heatingkwh], fillrange=zeros(24), ylim=(0,15), yticks=0:5:15, xticks=0:6:24, 
        color =[:firebrick], label=["d_fh"], legend=false, alpha=0.2);
    # bars
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] groupedbar!(
        [:B_HP :GR_HP :PV_HP], color =[:purple :grey :gold], 
        ylim=(0,15), yticks=0:5:15, xticks=0:6:24,
        annotations=(2, 14, text("$(Dates.month(date))/$(Dates.day(date))", 10)), legendfontsize=6, 
        label=["B_DE" "GR_DE" "PV_DE"], legend=false, bar_position = :stack, alpha=0.8);
    
    plot!(0:23, ones(24,1)*3, linestyle=:dot, linewidth= 2, color=:grey)
end

bar_heat (generic function with 2 methods)

In [41]:
function bar_comfort(Data_df, date)
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] groupedbar(
        ([:HP_FH :HP_HW].>0)*200, color =[:firebrick :steelblue], 
        ylim=(19,23), yticks=20:1:22, xticks=0:6:24,
        annotations=(2, 22.5, text("$(Dates.month(date))/$(Dates.day(date))", 10)), legendfontsize=6, 
        label=["Mod_fh" "Mod_hw"], legend=false, bar_position = :stack, alpha=0.15);
    
    # state-of-charge fh
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] plot!(
        [:Temp_FH], ylim=(19,23), yticks=20:1:22, xticks=0:6:24, color =[:firebrick], 
        label=["T_fh"], legend=false, linewidth= 2.0);
    
    plot!(0:23, ones(24,1)*20, linestyle=:dot, linewidth= 2, color=:firebrick)
    plot!(0:23, ones(24,1)*22, linestyle=:dot, linewidth= 2, color=:firebrick)
    
    plt = twinx()
    # state-of-charge hw
    @df Data_df[(Data_df[:,:day].==Dates.day(date)) .&(Data_df[:,:month].==Dates.month(date)),:] plot!(
        plt, [:Vol_HW], ylim=(10,190), yticks=20:40:180, xticks=0:6:24, color =[:steelblue], 
        label=["V_fh"], legend=false, linewidth= 2.0, linestyle=:dash, grid=false);
    
    plot!(plt, 0:23, ones(24,1)*20, linestyle=:dot, linewidth= 2, color=:steelblue)
    plot!(plt, 0:23, ones(24,1)*180, linestyle=:dot, linewidth= 2, color=:steelblue)
end

bar_comfort (generic function with 2 methods)

In [12]:
function bar_row(plotfunction, Data_df, date, length)
    plot([plotfunction(Data_df, (date+Day(i-1))) for i in 1:length]..., layout=(1,length), size=(300*length,200));
end

bar_row (generic function with 1 method)

In [62]:
@manipulate for d=Date(2015,1,1):Day(1):Date(2015,12,31), 
    Days=1:1:8
    
    p1=bar_row(bar_PV, Data_df, d, Days);
    p2=bar_row(bar_demand, Data_df, d, Days);
    p3=bar_row(bar_heat, Data_df, d, Days);
    p4=bar_row(bar_comfort, Data_df, d, Days);
    
    vbox(vskip(1em),
        md"*Interactive illustration of peer SHEMS results:*",
        vskip(1em),
        plot(p1,p2,p3,p4, layout=(4,1), size=(Days*300,4*200)))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["d"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 365,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
183, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Lissy\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Lissy\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Lissy\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Lissy\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Lissy\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000033f99150, Task (runnable) @0x0000000033f99150), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"2015-01-01\",\"2015-01-02\",\"2015-01-03\",\"2015-01-04\",\"2015-01-05\",\"2015-01-06\",\"2015-01-07\",\"2015-01-08\",\"2015-01-09\",\"2015-01-10\",\"2015-01-11\",\"2015-01-12\",\"2015-0